# word2vec: How To Prep Word Vectors For Modeling

### Train Our Own Model

In [2]:
# Read in the data, clean it, split it into train and test sets, and then train a word2vec model
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]


messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

### Prep Word Vectors

In [10]:
# Generate a list of words the word2vec model learned word vectors for
w2v_model.wv.index_to_key

['you',
 'to',
 'the',
 'and',
 'in',
 'is',
 'me',
 'my',
 'it',
 'for',
 'your',
 'of',
 'call',
 'have',
 'that',
 'on',
 'now',
 'are',
 'can',
 'but',
 'so',
 'not',
 'we',
 'or',
 'get',
 'do',
 'at',
 'ur',
 'if',
 'with',
 'will',
 'no',
 'just',
 'be',
 'this',
 'how',
 'gt',
 'lt',
 'up',
 'out',
 'free',
 'from',
 'what',
 'go',
 'ok',
 'when',
 'all',
 'll',
 'know',
 'like',
 'got',
 'am',
 'then',
 'come',
 'there',
 'he',
 'day',
 'its',
 'good',
 'was',
 'only',
 'time',
 'love',
 'send',
 'text',
 'as',
 'txt',
 'want',
 'one',
 'by',
 'sorry',
 'going',
 'home',
 'back',
 'still',
 'need',
 'about',
 'today',
 'stop',
 'see',
 'she',
 'don',
 'lor',
 'da',
 'dont',
 'our',
 'tell',
 'mobile',
 'later',
 'reply',
 'take',
 'did',
 'please',
 'they',
 'some',
 'think',
 'phone',
 'here',
 'pls',
 'hi',
 'new',
 'any',
 'claim',
 'her',
 'has',
 'much',
 'been',
 'night',
 'ì_',
 'well',
 'who',
 'more',
 're',
 'week',
 'dear',
 'great',
 'msg',
 'where',
 'him',
 'hey'

In [15]:
# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
w2v_vect = np.array([np.array([w2v_model.wv[word] for word in ls if word in w2v_model.wv.index_to_key])
                     for ls in X_test], dtype=object)

In [26]:
# # Why is the length of the sentence different than the length of the sentence vector?
for i, v in enumerate(w2v_vect):
    print(len(X_test.iloc[i]), len(v))

8 7
23 23
60 53
20 17
9 9
22 21
22 21
5 5
22 20
15 15
12 9
8 5
19 19
20 19
8 7
11 11
11 9
28 26
13 10
4 4
16 14
4 3
4 4
45 39
3 1
6 6
6 5
23 19
8 8
8 5
9 5
7 7
25 21
6 5
6 6
27 27
10 8
8 7
13 13
23 22
6 6
7 7
20 17
7 6
20 20
23 23
19 19
18 18
6 6
15 13
2 2
6 6
15 15
7 7
6 6
30 23
11 9
12 12
13 12
9 9
15 15
5 4
12 11
10 10
4 3
14 10
4 4
8 6
32 31
4 4
26 19
1 1
65 59
24 22
11 11
6 4
12 10
8 7
8 8
15 15
4 3
7 6
14 12
12 12
83 63
5 5
8 7
20 16
9 9
9 8
21 21
37 31
15 13
7 5
18 17
5 4
10 10
18 14
6 6
13 11
4 4
18 18
24 24
19 14
7 7
9 9
5 4
11 10
13 12
11 11
6 6
9 9
20 18
16 16
10 10
14 10
6 5
6 6
29 27
12 12
19 14
16 15
12 12
22 21
7 7
9 9
4 2
24 20
4 4
4 3
13 11
5 5
7 7
18 18
7 7
11 11
6 6
21 18
25 22
27 24
5 3
16 12
6 5
11 11
1 1
11 11
5 4
43 37
7 4
7 7
3 3
24 21
9 7
21 17
6 6
16 14
13 13
13 11
5 5
20 20
19 17
6 6
7 6
7 7
8 8
4 4
28 27
22 22
4 2
9 7
1 1
6 6
22 18
26 22
17 16
8 8
6 6
18 17
14 13
3 3
6 6
6 5
13 7
11 9
20 19
10 10
6 6
21 19
9 9
7 7
18 18
25 21
16 16
23 23
14 14
6 5
30 29
24 2

In [24]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
w2v_vect_avg = []

for vect in w2v_vect:
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [25]:
# Are our sentence vector lengths consistent?
for i, v in enumerate(w2v_vect_avg):
    print(len(X_test.iloc[i]), len(v))

8 100
23 100
60 100
20 100
9 100
22 100
22 100
5 100
22 100
15 100
12 100
8 100
19 100
20 100
8 100
11 100
11 100
28 100
13 100
4 100
16 100
4 100
4 100
45 100
3 100
6 100
6 100
23 100
8 100
8 100
9 100
7 100
25 100
6 100
6 100
27 100
10 100
8 100
13 100
23 100
6 100
7 100
20 100
7 100
20 100
23 100
19 100
18 100
6 100
15 100
2 100
6 100
15 100
7 100
6 100
30 100
11 100
12 100
13 100
9 100
15 100
5 100
12 100
10 100
4 100
14 100
4 100
8 100
32 100
4 100
26 100
1 100
65 100
24 100
11 100
6 100
12 100
8 100
8 100
15 100
4 100
7 100
14 100
12 100
83 100
5 100
8 100
20 100
9 100
9 100
21 100
37 100
15 100
7 100
18 100
5 100
10 100
18 100
6 100
13 100
4 100
18 100
24 100
19 100
7 100
9 100
5 100
11 100
13 100
11 100
6 100
9 100
20 100
16 100
10 100
14 100
6 100
6 100
29 100
12 100
19 100
16 100
12 100
22 100
7 100
9 100
4 100
24 100
4 100
4 100
13 100
5 100
7 100
18 100
7 100
11 100
6 100
21 100
25 100
27 100
5 100
16 100
6 100
11 100
1 100
11 100
5 100
43 100
7 100
7 100
3 100
24 100
9 100